In [1]:
!apt-get install unzip
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-23+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
--2020-08-17 14:04:39--  http://dataminingconsultant.com/DKD2e_data_sets.zip
Resolving dataminingconsultant.com (dataminingconsultant.com)... 160.153.91.162
Connecting to dataminingconsultant.com (dataminingconsultant.com)|160.153.91.162|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1003616 (980K) [application/zip]
Saving to: ‘DKD2e_data_sets.zip’

DKD2e_data_sets.zip 100%[===================>] 980.09K  2.33MB/s    in 0.4s    

2020-08-17 14:04:39 (2.33 MB/s) - ‘DKD2e_data_sets.zip’ saved [1003616/1003616]

Archive:  DKD2e_data_sets.zip
 extracting: Data sets/adult.zip     
  inflating: Data sets/cars.txt      
  inflating: Data sets/cars2.txt     
  inflating: Data sets/cereals.CSV   
  inflating: Data sets/churn.txt     
  inflating: Data sets/Cla

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('Data sets/churn.txt')
df.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


# create training and testing data set

In [4]:
train_data = df.sample(frac=0.8,random_state=200)
test_data = df.drop(train_data.index)
test_data_no_target = test_data.drop(columns=['Churn?'])

In [5]:
print('total:{} training:{} testing:{}'.format(df.shape[0], train_data.shape[0], test_data.shape[0]))

total:3333 training:2666 testing:667


# Upload data to s3

In [6]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-churn-prediction'

role = get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

In [7]:
train_data.to_csv('automl-train.csv', index=False, header=True) # Make sure features are comma-separated

In [9]:
train_data_url = session.upload_data(path='automl-train.csv', bucket=bucket, key_prefix=prefix + '/input')
train_data_url

's3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/input/automl-train.csv'

In [10]:
test_data.to_csv('test_data.csv', index=False, header=True)
test_data_no_target.to_csv('test_data_no_target.csv', index=False, header=True)

In [11]:
test_data_no_target_url = session.upload_data(path='test_data_no_target.csv', bucket=bucket, key_prefix=prefix + '/test')
test_data_no_target_url

's3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/test/test_data_no_target.csv'

In [12]:
# # verfiy the automl-train.csv
# automl_train = pd.read_csv('automl-train.csv')
# automl_train.head()

!aws s3 ls {train_data_url}

2020-08-17 14:06:05     250571 automl-train.csv


# Configure data location and artifacts

* Select target attribute "Churn?" to predict fraud or not fraud

In [13]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/input'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'Churn?'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

input_data_config

[{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
    'S3Uri': 's3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/input'}},
  'TargetAttributeName': 'Churn?'}]

In [14]:
output_data_config

{'S3OutputPath': 's3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/output'}

In [15]:
automl_job_config = {
  "CompletionCriteria": {
    "MaxCandidates": 10,
    "MaxRuntimePerTrainingJobInSeconds": 10*60
#       "MaxAutoMLJobRuntimeInSeconds": 60*120
  }
}
automl_job_config

{'CompletionCriteria': {'MaxCandidates': 10,
  'MaxRuntimePerTrainingJobInSeconds': 600}}

# Create SageMaker Auto-pilot job

In [16]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-churn-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=automl_job_config,
                      RoleArn=role)

AutoMLJobName: automl-churn-17-14-06-58


{'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:937236738569:automl-job/automl-churn-17-14-06-58',
 'ResponseMetadata': {'RequestId': '26bae063-1859-469d-8942-9d7b0742c2a3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '26bae063-1859-469d-8942-9d7b0742c2a3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Mon, 17 Aug 2020 14:06:59 GMT'},
  'RetryAttempts': 0}}

# Track Autopolit Job Status

In [17]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    job_secondary_status = describe_response['AutoMLJobSecondaryStatus']
    print (job_run_status + " - " + job_secondary_status)
    sleep(30)

JobStatus - Secondary Status
------------------------------
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - AnalyzingData
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - FeatureEngineering
InProgress - Featur

# Result

In [18]:
import pprint
import json
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']

best_candidate_name = best_candidate['CandidateName']
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{   'CandidateName': 'tuning-job-1-e1e9c554d7c14f40aa-001-48d934f8',
    'CandidateStatus': 'Completed',
    'CandidateSteps': [   {   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:937236738569:processing-job/db-1-84d989bc17734a278caa1e908b183e040348a49e12634be78bbcb8b895',
                              'CandidateStepName': 'db-1-84d989bc17734a278caa1e908b183e040348a49e12634be78bbcb8b895',
                              'CandidateStepType': 'AWS::SageMaker::ProcessingJob'},
                          {   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:937236738569:training-job/automl-chu-dpp4-1-046ffb698c5848caa69baf31be8aaafd7dd1e1e1b3204',
                              'CandidateStepName': 'automl-chu-dpp4-1-046ffb698c5848caa69baf31be8aaafd7dd1e1e1b3204',
                              'CandidateStepType': 'AWS::SageMaker::TrainingJob'},
                          {   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:937236738569:transform-job/automl-chu-dpp4-rpb-1-fede2df2ffcc4066bbd

# Create Model for best candidates

In [19]:
model_name = 'automl-churn-model-' + timestamp_suffix

model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))


Model ARN corresponding to the best candidate is : arn:aws:sagemaker:us-west-2:937236738569:model/automl-churn-model-17-14-06-58


# Use Transformation Job to test the model

In [20]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
transform_job_name = 'automl-churn-transform-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': test_data_no_target_url
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources
)

{'TransformJobArn': 'arn:aws:sagemaker:us-west-2:937236738569:transform-job/automl-churn-transform-17-14-43-51',
 'ResponseMetadata': {'RequestId': '398e31a1-d018-4433-9633-976ed8939cf1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '398e31a1-d018-4433-9633-976ed8939cf1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '111',
   'date': 'Mon, 17 Aug 2020 14:43:51 GMT'},
  'RetryAttempts': 0}}

# Get Batch Transform Job Status

In [21]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
Completed


# View Results of Transform Job

In [22]:
s3_output_path = sm.describe_transform_job(TransformJobName = transform_job_name)['TransformOutput']['S3OutputPath']
s3_output_key = s3_output_path + '/{}'.format('test_data_no_target.csv.out')
s3_output_key

's3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/inference-results/test_data_no_target.csv.out'

In [23]:
local_inference_results_path = 'inference_results.csv'
!aws s3 cp {s3_output_key} {local_inference_results_path}

download: s3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/inference-results/test_data_no_target.csv.out to ./inference_results.csv


In [24]:
data = pd.read_csv(local_inference_results_path, sep=';')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,True.
0,False.
1,False.
2,True.
3,False.
4,False.
...,...
662,False.
663,False.
664,False.
665,False.


In [25]:
test_data.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
17,VT,93,510,386-2923,no,no,0,190.7,114,32.42,...,111,18.55,129.6,121,5.83,8.1,3,2.19,3,False.
18,VA,76,510,356-2992,no,yes,33,189.7,66,32.25,...,65,18.09,165.7,108,7.46,10.0,5,2.70,1,False.
48,ID,119,415,398-1294,no,no,0,159.1,114,27.05,...,117,19.66,143.2,91,6.44,8.8,3,2.38,5,True.
67,MN,113,408,365-9011,yes,no,0,125.2,93,21.28,...,119,17.54,129.3,139,5.82,8.3,8,2.24,0,False.
73,NV,90,415,399-4246,no,no,0,203.4,146,34.58,...,117,19.27,152.4,105,6.86,7.3,4,1.97,1,False.


In [26]:
df_test_data_label = test_data['Churn?']
test_labels = df_test_data_label
test_labels.size

667

In [27]:
df_preds = pd.read_csv(local_inference_results_path, sep=';')
test_preds = df_preds['True.']
test_preds.size

667

In [28]:
accuracy = (test_labels.reset_index()['Churn?'] == test_preds[0]).sum() / len(test_labels)
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.8575712143928036


# View other candidates explored by SageMaker Autopilot

In [29]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
  print (str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))
  index += 1

1  tuning-job-1-e1e9c554d7c14f40aa-001-48d934f8  0.8991000056266785
2  tuning-job-1-e1e9c554d7c14f40aa-002-2c7ffcc1  0.8884699940681458
3  tuning-job-1-e1e9c554d7c14f40aa-009-87a459c4  0.7525399923324585
4  tuning-job-1-e1e9c554d7c14f40aa-008-2c67bf7f  0.7525399923324585
5  tuning-job-1-e1e9c554d7c14f40aa-005-ecca8f72  0.7525399923324585
6  tuning-job-1-e1e9c554d7c14f40aa-007-c87496b1  0.7392200231552124
7  tuning-job-1-e1e9c554d7c14f40aa-006-fac2e605  0.7392200231552124
8  tuning-job-1-e1e9c554d7c14f40aa-010-23eaef2b  0.7392200231552124
9  tuning-job-1-e1e9c554d7c14f40aa-004-8d07acad  0.7184500098228455
10  tuning-job-1-e1e9c554d7c14f40aa-003-0d711b92  0.4025973975658417


# Candidate Generation Notebook

In [30]:
nb_data_explore_url = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']
nb_data_explore_url

's3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/output/automl-churn-17-14-06-58/sagemaker-automl-candidates/pr-1-9fee4a8e68004cb58744370b080d43baefd3db73223243c395a514f304/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb'

In [31]:
nb_training_url = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']
nb_training_url

's3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/output/automl-churn-17-14-06-58/sagemaker-automl-candidates/pr-1-9fee4a8e68004cb58744370b080d43baefd3db73223243c395a514f304/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb'

In [32]:
!aws s3 cp {nb_data_explore_url} ./data_explore.ipynb
!aws s3 cp {nb_training_url} ./autopilot_training.ipynb

download: s3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/output/automl-churn-17-14-06-58/sagemaker-automl-candidates/pr-1-9fee4a8e68004cb58744370b080d43baefd3db73223243c395a514f304/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb to ./data_explore.ipynb
download: s3://sagemaker-us-west-2-937236738569/sagemaker/autopilot-churn-prediction/output/automl-churn-17-14-06-58/sagemaker-automl-candidates/pr-1-9fee4a8e68004cb58744370b080d43baefd3db73223243c395a514f304/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb to ./autopilot_training.ipynb
